In [ ]:
!pip install semantic-kernel==1.17.0 python-dotenv aiofiles nest_asyncio azure-search-documents pyperclip


## Import and Setup

In [19]:
import asyncio
import os
from pathlib import Path

import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import KernelFunctionSelectionStrategy
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import KernelFunctionTerminationStrategy
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt

load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-01-preview")


## Prepare Mock CSV Data (Simulated Files)
Here, we create local CSV files representing the insurance datasets.

In [24]:
from pathlib import Path

DATA_DIR = Path.cwd() / "insurance_data"
DATA_DIR.mkdir(exist_ok=True)

# Write CSV files
insurance_claims_content = """months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
328,48,521585,10/17/2014,OH,250/500,1000,1406.91,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,1/25/2015,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,9935 4th Drive,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
228,42,342868,6/27/2006,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,1/21/2015,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
134,29,687698,9/6/2000,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,2/22/2015,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7121 Francis Lane,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
256,41,227811,5/25/1990,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,1/10/2015,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
"""

(policy_data, fraud_rules, customer_faq, underwriting_guidelines) = (
"""policy_number,policy_state,coverage_type,coverage_limit,deductible,endorsements,flood_coverage,liability_limit,policy_start_date,policy_end_date
521585,OH,Homeowner,300000,1000,"{fire:yes,water:yes}",no,100000,2014-10-17,2025-10-17
342868,IN,Auto,250000,2000,"{theft:yes}",no,500000,2006-06-27,2026-06-27
687698,OH,Homeowner,350000,2000,"{fire:yes,water:yes}",yes,100000,2000-09-06,2030-09-06
227811,IL,Homeowner,300000,2000,"{fire:yes}",no,100000,1990-05-25,2030-05-25
""",
"""rule_id,pattern_description,trigger_condition,action
1,"Multiple small claims in 6 months","num_small_claims_6mo>2","flag_suspicious"
2,"Inconsistent incident date vs. report date","date_mismatch==True","request_additional_docs"
3,"Suspicious theft claims without police report","theft_claim==True AND police_report=='NO'","flag_suspicious"
4,"Frequent total loss collisions at odd hours","nighttime_collision==True AND total_loss==True","flag_suspicious"
""",
"""question_id,question_text,answer_text
101,"How do I file a claim?","Log into your BlueSky portal, select 'File a Claim', and follow the steps."
102,"Does my homeowner’s policy cover flood damage?","Flood damage requires a separate flood endorsement unless stated otherwise."
103,"What documents do I need for a major damage claim?","Professional repair estimates, invoices for replaced items, and if applicable, a police report."
""",
"""guideline_id,criterion,threshold_or_condition,required_action
U1,"High-value claims","claim_amount>25000","escalate_to_senior_underwriter"
U2,"Flood coverage requirement","flood_claim==True AND flood_coverage==False","deny_claim"
U3,"Require professional estimate","claim_amount>5000","request_professional_estimate"
U4,"Missing police report on theft","theft_claim==True AND police_report=='NO'","request_additional_docs"
"""
)

(DATA_DIR / "insurance_claims.csv").write_text(insurance_claims_content, encoding='utf-8')
(DATA_DIR / "policy_data.csv").write_text(policy_data, encoding='utf-8')
(DATA_DIR / "fraud_rules.csv").write_text(fraud_rules, encoding='utf-8')
(DATA_DIR / "customer_faq.csv").write_text(customer_faq, encoding='utf-8')
(DATA_DIR / "underwriting_guidelines.csv").write_text(underwriting_guidelines, encoding='utf-8')

vector_filenames = [
    str(DATA_DIR / "insurance_claims.csv"),
    str(DATA_DIR / "policy_data.csv"),
    str(DATA_DIR / "fraud_rules.csv"),
    str(DATA_DIR / "customer_faq.csv"),
    str(DATA_DIR / "underwriting_guidelines.csv")
]

# Convert all CSV files in vector_filenames to TXT
converted_filenames = []
for filename in vector_filenames:
    if filename.lower().endswith(".csv"):
        txt_filename = filename.replace(".csv", ".txt")
        with open(filename, "r", encoding="utf-8") as csv_file:
            content = csv_file.read()
        with open(txt_filename, "w", encoding="utf-8") as txt_file:
            txt_file.write(content)
        converted_filenames.append(txt_filename)
    else:
        # If it's already a supported format (like .txt), just add it
        converted_filenames.append(filename)

# Now converted_filenames will contain .txt versions instead of .csv
# You can use converted_filenames with your agent setup.
print("Converted files:")
for f in converted_filenames:
    print(f)


Converted files:
c:\Dev\azure-ai-search-python-playground\insurance_data\insurance_claims.txt
c:\Dev\azure-ai-search-python-playground\insurance_data\policy_data.txt
c:\Dev\azure-ai-search-python-playground\insurance_data\fraud_rules.txt
c:\Dev\azure-ai-search-python-playground\insurance_data\customer_faq.txt
c:\Dev\azure-ai-search-python-playground\insurance_data\underwriting_guidelines.txt


In [22]:
###################################################################
# 2. Kernel & Agent Setup
###################################################################
def create_kernel():
    kernel = Kernel()
    kernel.add_service(
        AzureChatCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )
    )
    return kernel

# Agent Names
COORDINATOR_NAME = "UnderwritingCoordinator"
FILE_SEARCHER_NAME = "FileSearcher"
DATA_ANALYST_NAME = "DataAnalyst"

# Coordinator agent: orchestrates underwriting decisions
coordinator_kernel = create_kernel()
agent_coordinator = ChatCompletionAgent(
    service_id=COORDINATOR_NAME,
    kernel=coordinator_kernel,
    name=COORDINATOR_NAME,
    instructions=f"""
        You are a senior Underwriting Coordinator Agent at BlueSky Insurance.
        The user will describe claim scenarios and ask for underwriting decisions.
        Your tasks:
        - Understand the user's request (e.g., assess a new claim, check coverage).
        - If you need data from policies, fraud rules, or guidelines, ask {FILE_SEARCHER_NAME}.
        - If you need analysis of claims history or fraud predictions, ask {DATA_ANALYST_NAME}.
        - After gathering data, provide a coherent underwriting recommendation.
        - If done, indicate satisfaction so the conversation ends.
    """,
)

In [ ]:
###################################################################
# 3. File Search Agent Creation (Known Issue: CSV not supported)
###################################################################
# Here is where the error is triggered: AzureAssistantAgent attempts to create a vector store 
# from CSV files, which is currently not supported by the API. 
# To debug:
# - Convert CSV files to .txt or another supported format before creating this agent
# - Or remove vector_store_filenames and disable file search
# Current code will fail with the "unsupported_file" error.

file_search_kernel = create_kernel()

try:
    file_search_agent = asyncio.run(AzureAssistantAgent.create(
        kernel=file_search_kernel,
        service_id=FILE_SEARCHER_NAME,
        name=FILE_SEARCHER_NAME,
        instructions="""
            You are a file-search agent. You have access to insurance-related CSV files (claims, policies, rules, FAQs, guidelines).
            Retrieve relevant information as requested by the coordinator. Return structured info.
        """,
        enable_file_search=True,
        vector_store_filenames=converted_filenames,  
        ai_model_id=AZURE_OPENAI_DEPLOYMENT,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        deployment_name=AZURE_OPENAI_DEPLOYMENT
    ))
except Exception as e:
    print("Error creating file_search_agent:", e)
    # DEBUG NOTE: The error indicates CSV files are not supported. 
    # To fix: Convert these CSV files to .txt or another supported format.
    file_search_agent = None

In [27]:
###################################################################
# 4. Data Analyst Agent Creation
###################################################################
# The Data Analyst agent uses code interpreter. This one should work if the environment is correct.
# But if the file_search_agent creation failed, this code still runs. You can conditionally skip it.

data_analyst_kernel = create_kernel()
try:
    data_analyst_agent = asyncio.run(AzureAssistantAgent.create(
        kernel=data_analyst_kernel,
        service_id=DATA_ANALYST_NAME,
        name=DATA_ANALYST_NAME,
        instructions="""
            You are a data analyst agent with code interpreter capability.
            Analyze claims data, check for fraud patterns, summarize claim amounts, etc.
            When asked, produce tables, metrics, or suggestions.
        """,
        enable_code_interpreter=True,
        code_interpreter_filenames=[], # no files attached
        ai_model_id=AZURE_OPENAI_DEPLOYMENT,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        deployment_name=AZURE_OPENAI_DEPLOYMENT
    ))
except Exception as e:
    print("Error creating data_analyst_agent:", e)
    data_analyst_agent = None

In [28]:
###################################################################
# 5. Group Chat Setup
###################################################################
# Only proceed if both file_search_agent and data_analyst_agent are successfully created.
if file_search_agent is not None and data_analyst_agent is not None:
    # Selection and Termination Strategies
    selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
        Determine which participant takes the next turn.
        Participants:
        - {COORDINATOR_NAME}
        - {FILE_SEARCHER_NAME}
        - {DATA_ANALYST_NAME}
        - User

        Rules:
        - After User speaks: {COORDINATOR_NAME} responds.
        - {COORDINATOR_NAME} may request {FILE_SEARCHER_NAME} for data or {DATA_ANALYST_NAME} for analysis.
        - After {FILE_SEARCHER_NAME} or {DATA_ANALYST_NAME} responds, go back to {COORDINATOR_NAME}.
        - Conversation ends when {COORDINATOR_NAME} is satisfied.
        History:
        {{{{$history}}}}
        """
    )

    termination_keyword = "done"
    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt=f"""
        If {COORDINATOR_NAME} shows a concluding or satisfied remark in the last message, return '{termination_keyword}'.
        Otherwise, do not.
        Response:
        {{{{$history}}}}
        """
    )

    group_chat_kernel = create_kernel()
    group_chat = AgentGroupChat(
        agents=[agent_coordinator, file_search_agent, data_analyst_agent],
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=group_chat_kernel,
            result_parser=lambda result: str(result.value[0]) if result.value else COORDINATOR_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_coordinator],
            function=termination_function,
            kernel=group_chat_kernel,
            result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
            history_variable_name="history",
            maximum_iterations=20,
        ),
    )

    print("Group chat created successfully!")
else:
    print("Group chat not created due to missing agents (file_search_agent or data_analyst_agent).")

Group chat created successfully!


## Create Agents and Group Chat

In [ ]:
def create_kernel():
    kernel = Kernel()
    kernel.add_service(
        AzureChatCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )
    )
    return kernel

# Agent Names
COORDINATOR_NAME = "UnderwritingCoordinator"
FILE_SEARCHER_NAME = "FileSearcher"
DATA_ANALYST_NAME = "DataAnalyst"

# Coordinator agent: orchestrates underwriting decisions
coordinator_kernel = create_kernel()
agent_coordinator = ChatCompletionAgent(
    service_id=COORDINATOR_NAME,
    kernel=coordinator_kernel,
    name=COORDINATOR_NAME,
    instructions=f"""
        You are a senior Underwriting Coordinator Agent at BlueSky Insurance.
        The user will describe claim scenarios and ask for underwriting decisions.
        Your tasks:
        - Understand the user's request (e.g., assess a new claim, check coverage).
        - If you need data from policies, fraud rules, or guidelines, ask {FILE_SEARCHER_NAME}.
        - If you need analysis of claims history or fraud predictions, ask {DATA_ANALYST_NAME}.
        - After gathering data, provide a coherent underwriting recommendation.
        - If done, indicate satisfaction so the conversation ends.
    """,
)

# File Searcher agent: retrieves info from CSVs
file_search_kernel = create_kernel()
file_search_agent = asyncio.run(AzureAssistantAgent.create(
    kernel=file_search_kernel,
    service_id=FILE_SEARCHER_NAME,
    name=FILE_SEARCHER_NAME,
    instructions="""
        You are a file-search agent. You have access to insurance-related CSV files (claims, policies, rules, FAQs, guidelines).
        Retrieve relevant information as requested by the coordinator. Return structured info.
    """,
    enable_file_search=True,
    vector_store_filenames=vector_filenames,
    ai_model_id=AZURE_OPENAI_DEPLOYMENT,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    deployment_name=AZURE_OPENAI_DEPLOYMENT
))

# Data Analyst agent: uses code interpreter for analysis
data_analyst_kernel = create_kernel()
data_analyst_agent = asyncio.run(AzureAssistantAgent.create(
    kernel=data_analyst_kernel,
    service_id=DATA_ANALYST_NAME,
    name=DATA_ANALYST_NAME,
    instructions="""
        You are a data analyst agent with code interpreter capability.
        Analyze claims data, check for fraud patterns, summarize claim amounts, etc.
        When asked, produce tables, metrics, or suggestions.
    """,
    enable_code_interpreter=True,
    code_interpreter_filenames=[], # If needed, we can attach CSVs for direct code analysis
    ai_model_id=AZURE_OPENAI_DEPLOYMENT,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
))

# Selection and Termination Strategies
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn.
    Participants:
    - {COORDINATOR_NAME}
    - {FILE_SEARCHER_NAME}
    - {DATA_ANALYST_NAME}
    - User

    Rules:
    - After User speaks: {COORDINATOR_NAME} responds.
    - {COORDINATOR_NAME} may request {FILE_SEARCHER_NAME} for data or {DATA_ANALYST_NAME} for analysis.
    - After {FILE_SEARCHER_NAME} or {DATA_ANALYST_NAME} responds, go back to {COORDINATOR_NAME}.
    - Conversation ends when {COORDINATOR_NAME} is satisfied.
    History:
    {{{{$history}}}}
    """
)

termination_keyword = "done"
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt=f"""
    If {COORDINATOR_NAME} shows a concluding or satisfied remark in the last message, return '{termination_keyword}'.
    Otherwise, do not.
    Response:
    {{{{$history}}}}
    """
)

group_chat_kernel = create_kernel()
group_chat = AgentGroupChat(
    agents=[agent_coordinator, file_search_agent, data_analyst_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=group_chat_kernel,
        result_parser=lambda result: str(result.value[0]) if result.value else COORDINATOR_NAME,
        agent_variable_name="agents",
        history_variable_name="history",
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[agent_coordinator],
        function=termination_function,
        kernel=group_chat_kernel,
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="history",
        maximum_iterations=20,
    ),
)


In [29]:
async def chat_loop():
    print("Ask your underwriting questions. Type 'exit' to quit.")
    is_complete = False
    while not is_complete:
        user_input = input("User:> ")
        if user_input.lower() == "exit":
            break

        if not user_input.strip():
            continue

        await group_chat.add_chat_message(
            ChatMessageContent(role=AuthorRole.USER, content=user_input)
        )

        async for response in group_chat.invoke():
            print(f"# {response.role} - {response.name or '*'}: '{response.content}'")

        if group_chat.is_complete:
            is_complete = True

    print("Conversation ended.")

await chat_loop()


Ask your underwriting questions. Type 'exit' to quit.


Failed to select agent: Agent Failure - Strategy unable to select next agent: The next participant to take the turn is the **UnderwritingCoordinator**.


AgentChatException: Failed to select agent